In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras.utils as image
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from keras import models
from keras import layers
from keras import optimizers
from PIL import Image
import sklearn
from sklearn.model_selection import train_test_split
from statistics import mean
from keras.applications import VGG16
import warnings
warnings.filterwarnings('ignore')

In [2]:
def arrange_data(df):
    
    image_data = []
    img_paths = np.asarray(df.iloc[:, 0]) #First column is the image paths
    
    for i in tqdm(range(len(img_paths))):       
        img = image.load_img(img_paths[i],target_size=(200,150,3))
        img = image.img_to_array(img)
        img = img/255
        image_data.append(img)
        
    X = np.array(image_data)
    Y = np.array(df.iloc[:,1:29])
    
    print("Shape of images:", X.shape)
    print("Shape of labels:", Y.shape)
    
    return X, Y

In [3]:
print("Processing train..")
train_df = pd.read_csv("Data/Train.csv", delimiter=" ")
X_train, Y_train = arrange_data (train_df)

print("Processing valid..")
val_df = pd.read_csv("Data/Valid.csv", delimiter=" ")
X_val, Y_val = arrange_data (val_df)

Processing train..


100%|█████████████████████████████████████████████████████████████████████████████| 4838/4838 [00:19<00:00, 252.86it/s]


Shape of images: (4838, 200, 150, 3)
Shape of labels: (4838, 28)
Processing valid..


100%|█████████████████████████████████████████████████████████████████████████████| 1866/1866 [00:07<00:00, 262.46it/s]


Shape of images: (1866, 200, 150, 3)
Shape of labels: (1866, 28)


In [4]:
num_classes = 28  

model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(200,150,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 196, 146, 16)      1216      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 98, 73, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 98, 73, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 94, 69, 32)        12832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 47, 34, 32)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 47, 34, 32)        0

In [5]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=30, validation_data=(X_val, Y_val), batch_size=64)
model.save('Models/Model_6c.h5')

Epoch 1/30
76/76 [==============================] - 107s 1s/step - loss: 0.4057 - accuracy: 0.1186 - val_loss: 0.2539 - val_accuracy: 0.2090
Epoch 2/30
76/76 [==============================] - 105s 1s/step - loss: 0.2758 - accuracy: 0.1895 - val_loss: 0.2354 - val_accuracy: 0.2090
Epoch 3/30
76/76 [==============================] - 108s 1s/step - loss: 0.2564 - accuracy: 0.1904 - val_loss: 0.2306 - val_accuracy: 0.2090
Epoch 4/30
76/76 [==============================] - 111s 1s/step - loss: 0.2483 - accuracy: 0.2024 - val_loss: 0.2307 - val_accuracy: 0.2090
Epoch 5/30
76/76 [==============================] - 110s 1s/step - loss: 0.2445 - accuracy: 0.1966 - val_loss: 0.2295 - val_accuracy: 0.2090
Epoch 6/30
76/76 [==============================] - 110s 1s/step - loss: 0.2414 - accuracy: 0.1955 - val_loss: 0.2301 - val_accuracy: 0.2090
Epoch 7/30
76/76 [==============================] - 112s 1s/step - loss: 0.2399 - accuracy: 0.1955 - val_loss: 0.2298 - val_accuracy: 0.2090
Epoch 8/30
76

In [4]:
num_classes = 28  # Types of Genres
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(200,150, 3))
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
    
model = models.Sequential()
model.add(vgg_conv)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 6, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 12288)             0         
                                                                 
 dense (Dense)               (None, 1024)              12583936  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 28)                28700     
                                                                 
Total params: 27,327,324
Trainable params: 19,692,060
Non-trainable params: 7,635,264
_________________________________________________________________


In [5]:
model.compile(optimizer=optimizers.RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,width_shift_range=0.2, height_shift_range=0.2, 
                         shear_range=0.15,horizontal_flip=True, fill_mode="nearest")

EPOCHS=10
BS = 64
history = model.fit_generator(aug.flow(X_train, Y_train, batch_size=BS),validation_data=(X_val, Y_val), 
                    steps_per_epoch=len(X_train) // BS, epochs=EPOCHS)
model.save('Models/Model_4d.h5')

Epoch 1/10
75/75 [==============================] - 1290s 17s/step - loss: 0.2656 - accuracy: 0.2038 - val_loss: 0.2323 - val_accuracy: 0.2583
Epoch 2/10
75/75 [==============================] - 1325s 18s/step - loss: 0.2417 - accuracy: 0.2128 - val_loss: 0.2301 - val_accuracy: 0.2701
Epoch 3/10
75/75 [==============================] - 1309s 18s/step - loss: 0.2359 - accuracy: 0.2250 - val_loss: 0.2280 - val_accuracy: 0.2192
Epoch 4/10
75/75 [==============================] - 1315s 18s/step - loss: 0.2321 - accuracy: 0.2289 - val_loss: 0.2268 - val_accuracy: 0.2379
Epoch 5/10
75/75 [==============================] - 1308s 17s/step - loss: 0.2300 - accuracy: 0.2350 - val_loss: 0.2290 - val_accuracy: 0.2128
Epoch 6/10
75/75 [==============================] - 1326s 18s/step - loss: 0.2276 - accuracy: 0.2419 - val_loss: 0.2278 - val_accuracy: 0.2288
Epoch 7/10
75/75 [==============================] - 1328s 18s/step - loss: 0.2243 - accuracy: 0.2547 - val_loss: 0.2276 - val_accuracy: 0.2363